In [1]:
import duckdb

In [19]:
con = duckdb.connect('/content/drive/MyDrive/github_projects/data_modeling/tabelas_sql/star_schema/star_schema.db')

# Verificar se as tabelas estão disponíveis
print(con.execute("SHOW TABLES").fetchall())

[('categorias_produto',), ('clientes',), ('geolocalizacao',), ('itens_pedido',), ('pagamentos',), ('pedidos',), ('produtos',), ('reviews_table',), ('vendedores',)]


In [27]:
# 1. Tabela dimensão: Clientes
con.execute("""
CREATE TABLE dim_clientes AS
SELECT DISTINCT
    customer_id,
    customer_unique_id,
    customer_city,
    customer_state
FROM clientes;
""")

# 2. Tabela dimensão: Produtos
con.execute("""
CREATE TABLE dim_produtos AS
SELECT DISTINCT
    product_id,
    product_category_name,
    product_name_lenght,
    product_description_lenght,
    product_photos_qty,
    product_weight_g,
    product_length_cm,
    product_width_cm
FROM produtos;
""")

# 3. Tabela dimensão: Vendedores
con.execute("""
CREATE TABLE dim_vendedores AS
SELECT DISTINCT
    seller_id,
    seller_zip_code_prefix,
    seller_city,
    seller_state
FROM vendedores;
""")

# 4. Tabela dimensão: Tempo
con.execute("""
CREATE TABLE dim_tempo AS
SELECT DISTINCT
    order_purchase_timestamp::TIMESTAMP AS data_pedido,
    EXTRACT(YEAR FROM order_purchase_timestamp::TIMESTAMP) AS ano,
    EXTRACT(MONTH FROM order_purchase_timestamp::TIMESTAMP) AS mes,
    EXTRACT(DAY FROM order_purchase_timestamp::TIMESTAMP) AS dia,
    EXTRACT(dow FROM order_purchase_timestamp::TIMESTAMP) AS dia_semana
FROM pedidos;
""")

# 5. Tabela fato: Pedidos (fatos numéricos)
con.execute("""
CREATE TABLE fato_pedidos AS
SELECT
    p.order_id,
    p.customer_id,
    oi.product_id,
    oi.seller_id,
    p.order_purchase_timestamp,
    SUM(oi.price) AS total_valor,
    SUM(oi.freight_value) AS total_frete,
    AVG(rev.review_score) AS media_avaliacao,
    SUM(pay.payment_value) AS total_pago
FROM pedidos p
JOIN itens_pedido oi ON p.order_id = oi.order_id
LEFT JOIN reviews_table rev ON p.order_id = rev.order_id
LEFT JOIN pagamentos pay ON p.order_id = pay.order_id
GROUP BY
    p.order_id, p.customer_id, oi.product_id, oi.seller_id, p.order_purchase_timestamp;
""")

print("✅ Star Schema criado com sucesso!")

✅ Star Schema criado com sucesso!


In [8]:
con.close()

In [29]:
print(con.execute("SHOW TABLES").fetchall())

[('categorias_produto',), ('clientes',), ('dim_clientes',), ('dim_produtos',), ('dim_tempo',), ('dim_vendedores',), ('fato_pedidos',), ('geolocalizacao',), ('itens_pedido',), ('pagamentos',), ('pedidos',), ('produtos',), ('reviews_table',), ('vendedores',)]


In [30]:
con.execute("""
DROP TABLE IF EXISTS categorias_produto;
DROP TABLE IF EXISTS clientes;
DROP TABLE IF EXISTS geolocalizacao;
DROP TABLE IF EXISTS itens_pedido;
DROP TABLE IF EXISTS pagamentos;
DROP TABLE IF EXISTS pedidos;
DROP TABLE IF EXISTS produtos;
DROP TABLE IF EXISTS reviews_table;
DROP TABLE IF EXISTS vendedores;
""")

In [61]:
## Tabela Fato: fato_pedidos

fato_pedidos_df = con.execute("""
SELECT *
FROM fato_pedidos
""").df()

display(fato_pedidos_df.head())

,order_id,customer_id,product_id,seller_id,order_purchase_timestamp,total_valor,total_frete,media_avaliacao,total_pago
0,e8805f1923e711c407c454831b49643f,9b5d68f669f7ed215f789471556529fe,80103e141ed0da3b1d63cb0c7d1f7d48,0afccdb8a34ee5c79f7c06faf2b4d56e,2017-04-05 11:04:33,99.90,14.87,5.0,114.77
1,e880a2a0f19dcb0f2ef51fed71ef34c9,5ca0847591cdad1a279fd89aa4e58e04,37116a322ceabbe82c30430000af0db3,d1c281d3ae149232351cd8c8cc885f0d,2018-03-29 01:10:03,35.98,38.64,5.0,149.24
2,e880c960b083af6aae64e809acf0693c,7c5df4a2ffa11a657198a578c83fd58f,461f43be3bdf8844e65b62d9ac2c7a5a,4869f7a5dfa277a7dca6462dcf3b52b2,2017-07-14 00:08:37,315.98,31.72,2.0,695.40
3,e883aa370777928f8939c79a58ef69b4,a930e8971b7eed8b6fd594e8bc933058,45b280868bcc8124d9309f459a42eeaf,4d6d651bd7684af3fffabd5f08d12e5a,2018-02-01 07:49:37,169.00,15.93,5.0,184.93
4,e8844ed350d0a0556e982b89e2d727cb,2b11be9617fad2924dfc186b73220db0,b9e88d457e276521ccb0b226f2f8f18f,3c487ae8f8d7542beff5788e2e0aea83,2018-02-28 19:16:03,385.90,27.98,5.0,413.88


In [62]:
## Tabela Dimensão: dim_clientes

dim_clientes_df = con.execute("""
SELECT *
FROM dim_clientes
""").df()

# Exibir o DataFrame
display(dim_clientes_df.head())

,customer_id,customer_unique_id,customer_city,customer_state
0,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,mogi das cruzes,SP
1,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,jaragua do sul,SC
2,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,timoteo,MG
3,9fb35e4ed6f0a14a4977cd9aea4042bb,2a7745e1ed516b289ed9b29c7d0539a5,montes claros,MG
4,206f3129c0e4d7d0b9550426023f0a08,21f748a16f4e1688a9014eb3ee6fa325,piracicaba,SP


In [63]:
## Tabela Dimensão: dim_produtos

dim_produtos_df = con.execute("""
SELECT *
FROM dim_produtos
""").df()

# Exibir o DataFrame
display(dim_produtos_df.head())

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_width_cm
0,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,15.0
1,8c92109888e8cdf9d66dc7e463025574,brinquedos,36.0,1156.0,1.0,600.0,17.0,12.0
2,03b63c5fc16691530586ae020c345514,bebes,49.0,728.0,4.0,7150.0,50.0,45.0
3,7bb6f29c2be57716194f96496660c7c2,moveis_decoracao,51.0,2083.0,2.0,600.0,68.0,13.0
4,3bb7f144022e6732727d8d838a7b13b3,esporte_lazer,22.0,3021.0,1.0,800.0,16.0,11.0


In [64]:
## Tabela Dimensão: dim_vendedores

dim_vendedores_df = con.execute("""
SELECT *
FROM dim_vendedores
""").df()

# Exibir o DataFrame
display(dim_vendedores_df.head())

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
1,8cb7c5ddf41f4d506eba76e9a4702a25,75110,anapolis,GO
2,e38db885400cd35c71dfd162f2c1dbcf,70740,brasilia,DF
3,d2e753bb80b7d4faa77483ed00edc8ca,45810,porto seguro,BA
4,430315b7bb4b6e4b3c978f9dfa9b0558,4857,sao paulo,SP


In [65]:
## Tabela Dimensão: dim_tempo

dim_tempo_df = con.execute("""
SELECT *
FROM dim_tempo
""").df()

# Exibir o DataFrame
display(dim_tempo_df.head())

,data_pedido,ano,mes,dia,dia_semana
0,2018-07-24 20:41:37,2018,7,24,2
1,2017-04-11 12:22:08,2017,4,11,2
2,2018-06-07 19:03:12,2018,6,7,4
3,2017-09-18 14:31:30,2017,9,18,1
4,2018-01-08 07:55:29,2018,1,8,1


##  1. Total de vendas por ano

In [66]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

con.execute("""
SELECT
    dt.ano,
    SUM(fp.total_valor) AS total_vendas
FROM fato_pedidos fp
JOIN dim_tempo dt
    ON CAST(fp.order_purchase_timestamp AS TIMESTAMP) = dt.data_pedido
GROUP BY dt.ano
ORDER BY dt.ano;
""").df()

,ano,total_vendas
0,2016,"51,347.18"
1,2017,"6,522,801.05"
2,2018,"7,699,551.42"


In [32]:
con.execute("PRAGMA table_info(fato_pedidos);").fetchdf()

,cid,name,type,notnull,dflt_value,pk
0,0,order_id,VARCHAR,False,None,False
1,1,customer_id,VARCHAR,False,None,False
2,2,product_id,VARCHAR,False,None,False
3,3,seller_id,VARCHAR,False,None,False
4,4,order_purchase_timestamp,VARCHAR,False,None,False
5,5,total_valor,DOUBLE,False,None,False
6,6,total_frete,DOUBLE,False,None,False
7,7,media_avaliacao,DOUBLE,False,None,False
8,8,total_pago,DOUBLE,False,None,False


## 2. produtos mais vendidos (por quantidade)

In [42]:
con.execute("""
SELECT
    dp.product_category_name,
    COUNT(*) AS total_vendido
FROM fato_pedidos fp
JOIN dim_produtos dp ON fp.product_id = dp.product_id
GROUP BY dp.product_category_name;

""").df()

,product_category_name,total_vendido
0,esporte_lazer,7858
1,moveis_decoracao,6781
2,market_place,284
3,brinquedos,3979
4,ferramentas_jardim,3607
...,...,...
69,artes_e_artesanato,24
70,cds_dvds_musicais,12
71,dvds_blu_ray,62
72,portateis_cozinha_e_preparadores_de_alimentos,14


##  3. Estados com mais pedidos

In [49]:
con.execute("""
SELECT
    dc.customer_state,
    COUNT(*) AS total_pedidos
FROM fato_pedidos fp
JOIN dim_clientes dc ON fp.customer_id = dc.customer_id
GROUP BY dc.customer_state
ORDER BY total_pedidos DESC;
""").df()

,customer_state,total_pedidos
0,SP,43052
1,RJ,13245
2,MG,11986
3,RS,5637
4,PR,5157
5,SC,3737
6,BA,3480
7,DF,2211
8,ES,2098
9,GO,2093


## 4. Formas de pagamento mais comuns

In [58]:
con.execute("""
SELECT
    'Pagamento' AS forma_pagamento,
    COUNT(*) AS total_transacoes,
    SUM(fp.total_pago) AS total_pago
FROM fato_pedidos fp
GROUP BY forma_pagamento;

""").df()


,forma_pagamento,total_transacoes,total_pago
0,Pagamento,102425,20416842.54


## 5. Média de Avaliação por Produto

In [56]:
con.execute("""
SELECT
    dp.product_category_name,
    AVG(fp.media_avaliacao) AS media_avaliacao
FROM fato_pedidos fp
JOIN dim_produtos dp ON fp.product_id = dp.product_id
GROUP BY dp.product_category_name
ORDER BY media_avaliacao DESC;
""").df()


,product_category_name,media_avaliacao
0,cds_dvds_musicais,4.666667
1,fashion_roupa_infanto_juvenil,4.500000
2,livros_interesse_geral,4.446768
3,construcao_ferramentas_ferramentas,4.425532
4,flores,4.392857
...,...,...
69,fashion_roupa_masculina,3.639344
70,moveis_escritorio,3.594354
71,portateis_cozinha_e_preparadores_de_alimentos,3.428571
72,pc_gamer,3.125000


In [59]:
con.execute("""
SELECT
*
FROM fato_pedidos
""").df()


,order_id,customer_id,product_id,seller_id,order_purchase_timestamp,total_valor,total_frete,media_avaliacao,total_pago
0,e8805f1923e711c407c454831b49643f,9b5d68f669f7ed215f789471556529fe,80103e141ed0da3b1d63cb0c7d1f7d48,0afccdb8a34ee5c79f7c06faf2b4d56e,2017-04-05 11:04:33,99.90,14.87,5.0,114.77
1,e880a2a0f19dcb0f2ef51fed71ef34c9,5ca0847591cdad1a279fd89aa4e58e04,37116a322ceabbe82c30430000af0db3,d1c281d3ae149232351cd8c8cc885f0d,2018-03-29 01:10:03,35.98,38.64,5.0,149.24
2,e880c960b083af6aae64e809acf0693c,7c5df4a2ffa11a657198a578c83fd58f,461f43be3bdf8844e65b62d9ac2c7a5a,4869f7a5dfa277a7dca6462dcf3b52b2,2017-07-14 00:08:37,315.98,31.72,2.0,695.40
3,e883aa370777928f8939c79a58ef69b4,a930e8971b7eed8b6fd594e8bc933058,45b280868bcc8124d9309f459a42eeaf,4d6d651bd7684af3fffabd5f08d12e5a,2018-02-01 07:49:37,169.00,15.93,5.0,184.93
4,e8844ed350d0a0556e982b89e2d727cb,2b11be9617fad2924dfc186b73220db0,b9e88d457e276521ccb0b226f2f8f18f,3c487ae8f8d7542beff5788e2e0aea83,2018-02-28 19:16:03,385.90,27.98,5.0,413.88
...,...,...,...,...,...,...,...,...,...
102420,e87746786779edef45938a6bc4572c0b,817a3322ad71f73a2bd444f27583b4e6,308a7360ee2e0ecfb6aa7708428692af,128639473a139ac0f3e5f5ade55873a5,2018-02-03 23:17:59,25.90,25.63,5.0,51.53
102421,e8794f8491ea025138cfb2005b927693,57753ca70682fa16e7780c6f0189d105,9c7ce59c96e74dfdfc51f13a4a6c3869,6560211a19b47992c3666cc44a7e94c0,2017-06-24 19:28:36,29.00,7.78,5.0,36.78
102422,e87aa2b0086ab1f74f1faa822186807c,422fe1142d7c4fcb0cfe90720b24163f,8f79239910bb0bf15488ef9abd9e6593,12b9676b00f60f3b700e83af21824c0e,2017-04-11 18:09:20,199.00,28.14,4.0,227.14
102423,e87d63870bcff2f72abfc64454bfff05,da5edefc151065bf935b9bf2c412e96b,10f2fea49b203930a1f4c0a3a892d960,638cba8be1fb599bbb76fd6948351eb3,2017-08-09 13:23:18,12.50,11.85,5.0,24.35
